In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
 
    

In [2]:
def output_corners():
    for i in range(1,21):
        inpath = "./camera_cal/calibration%d.jpg" % i
        outpath = "./output_images/corners%d.jpg" % i
        
        nx = 9
        ny = 5 if i in [1, 4, 5] else 6
        img = cv2.imread(inpath)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if ret:
            cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            cv2.imwrite(outpath, img)


In [3]:
def catalog_images():
    imgs = []
    for i in range(1, 4):
        inf = {}
        inf["path"] = "./camera_cal/calibration%d.jpg" % i
        inf["undist_path"] = "./output_images/calibration_undist%d.jpg" % i
        inf["nx"] = 9
        inf["ny"] = 5 if i == 1 else 6
        imgs.append(inf)
    return imgs

def calculate_undist():
    # make objpoints
    # find corners
    # calculate distortion
    # undistort

    imgs = catalog_images()
    objpoints = []
    imgpoints = []

    for inf in imgs:
        nx = inf["nx"]
        ny = inf["ny"]
        imgpath = inf["path"]
    
        objp = np.zeros((nx*ny,3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
        
        img = cv2.imread(inf["path"])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        img_size = gray.shape[::-1]
        if img_size != (1280, 720):
            continue

        ret, imgp = cv2.findChessboardCorners(gray, (nx, ny), None)

        if not ret:
            continue
            
        objpoints.append(objp)
        imgpoints.append(imgp)
        
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp], [imgp], gray.shape[::-1], None, None)
    
    for inf in imgs:
        nx = inf["nx"]
        ny = inf["ny"]
        imgpath = inf["path"]
        img = cv2.imread(inf["path"])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        if img_size != (1280, 720):
            continue
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(inf["undist_path"], undist)
        
    return ret, mtx, dist, rvecs, tvecs


def test_images():
    return ["./test_images/straight_lines%d.jpg" % i for i in [1, 2]] + \
           ["./test_images/test%d.jpg" % i for i in range(1,7)]
    
def show_undist():
    ret, mtx, dist, rvecs, tvecs = calculate_undist()
    ti = test_images()
    for path in ti:
        img = cv2.imread(path)
        try:
            undist = cv2.undistort(img, mtx, dist, None, mtx)
        except:
            print(path)
            print(img)
            
        outpath = path.replace("test_images/", "output_images/undist_")
        cv2.imwrite(outpath, undist)
      



In [4]:
show_undist()

In [5]:
output_corners()    
    